# Einführungsbeispiel - Zweifeldträger

Das Vorlesungsbeispiel verfolgt das Ziel, die Verformungen eines Zweifeldträgers für zwei Betonstähle mit unterschiedlichen Duktilitätsklassen aufzuzeigen. @jager_stahlbeton_2009.

![Statisches System des Zweifeldträgers mit Abmessungen versehen, daneben der Querschnitt mit Angaben zu der Längsbewehrung](../imgs/zweifeld_system.png){#fig-zweifeld_system}

Zunächst wird die analytische Lösung vollumfänglich dargestellt und mit numerischen Werten substituiert. Abschliessend wird das Verformungsverhalten mit dem Federmodell nachmodelliert, bzw. numerisch approximiert.

## Allgemeines

### Annahmen

- Zwei unterschiedliche Stähle
- konstant gerissene Biegesteifigkeit entlang der Stabachse

### Parameter



In [1]:
_

from packages import *

#### Bewehrung

In [2]:
# Längsbewehrung in X-Richtung gerichtet
diam_x = 14 * mm

# Teilung der Längsbewehrung
s_x = 200 * mm

# Längsbewehrung in Y-Richtung gerichtet
diam_y = 12 * mm

# Teilung der Längsbewehrung
s_y = 200 * mm

# Querschnittsfläche der Längsbewehrung, pro Längeneinheit
a_s = (diam_x**2 / 4 * np.pi / s_x).to(mm**2 / m)

put_out()

$$
\begin{aligned}
\oslash_{x} & = 14 \ \mathrm{mm} \quad & s_{x} & = 200 \ \mathrm{mm} \quad & \oslash_{y} & = 12 \ \mathrm{mm} \\ 
s_{y} & = 200 \ \mathrm{mm} \quad & a_{s} & = 769.69 \ \frac{\mathrm{mm}^{2}}{\mathrm{m}} \quad &  
 \end{aligned}
$$

#### Geometrie

In [3]:
# Betonüberdeckung
c = 20 * mm

# Plattenstärke
h = 300 * mm

# Streifenbreite
b_w = 1 * m

# Balkenlänge
l = 8 * m

# Statische Höhe in der Bewehrung in X-Richtung
d_x = h - c - diam_y - 1 / 2 * diam_x

put_out()

$$
\begin{aligned}
c & = 20 \ \mathrm{mm} \quad & h & = 300 \ \mathrm{mm} \quad & b_{w} & = 1 \ \mathrm{m} \\ 
l & = 8 \ \mathrm{m} \quad & d_{x} & = 261.0 \ \mathrm{mm} \quad &  
 \end{aligned}
$$

#### Beton

In [4]:
# Zylinderdruckfestigkeit vorausgesetzt
f_cc = 30 * N / mm**2

# Betonzugfestigkeit, nicht einheitentreu
f_ct = (0.3 * f_cc.m ** (2 / 3)) * N / mm**2

# Effektive Betondruckfestigkeit, nicht einheitentreu
f_c = (2.5 * f_cc.m ** (2 / 3)) * N / mm**2

# Elastizitätsmodul des Betons
E_c = 29.3 * kN / mm**2

# Bruchstauchung Erfahrungswert
varepsilon_cu = 5 * permille

put_out()

$$
\begin{aligned}
f_{cc} & = 30.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & f_{ct} & = 2.9 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & f_{c} & = 24.14 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \\ 
E_{c} & = 29.3 \ \frac{\mathrm{kN}}{\mathrm{mm}^{2}} \quad & \varepsilon_{cu} & = 5 \ \mathrm{‰} \quad &  
 \end{aligned}
$$

### Modellbildung - Zuggurtmodell

In [5]:
# Verbundschubspannung vor dem Fliessen
tau_b0 = 2 * f_ct

# Verbundschubspannung vor dem Fliessen
tau_b1 = f_ct

put_out()

$$
\begin{aligned}
\tau_{b0} & = 5.79 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & \tau_{b1} & = 2.9 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \end{aligned}
$$

In [6]:
# Abschätzung der mitwirkenden Höhe der Betonzugzone
h_eff = 2 * (h - d_x)

# Geometrischer Bewehrungsgehalt
rho = (a_s / h_eff).to(percent)

put_out()

$$
\begin{aligned}
h_{eff} & = 78.0 \ \mathrm{mm} \quad & \rho & = 0.99 \ \mathrm{\%} \end{aligned}
$$

In [7]:
# maximaler Rissabstand
s_r0 = (diam_x * (1 - rho) / (4 * rho)).to(mm)

# minimaler Rissabstand
s_rl = 1 / 2 * s_r0

# Annahme des Rissabstandes, entspricht dem Abstand der Querstäbe
s_r = 200 * mm

put_out()

$$
\begin{aligned}
s_{r0} & = 351.19 \ \mathrm{mm} \quad & s_{rl} & = 175.59 \ \mathrm{mm} \quad & s_{r} & = 200 \ \mathrm{mm} \end{aligned}
$$

In [8]:
# Bestimmung der mittleren Stahldehnung

def esm_bilinear(
    E_s, E_sh, s_r, tau_b0, tau_b1, diam, sigma_sr, f_sy, f_su, epsilon_sy
):
    display(Markdown("Berechnungen nach @marti_tragverhalten_1999 Seite 156"))
    if sigma_sr <= f_sy:
        display(Markdown("Gleichung 1"))
        return sigma_sr / E_s - tau_b0 * s_r / (E_s * diam)

    if f_sy <= sigma_sr <= (f_sy + (2 * tau_b1 * s_r) / diam):
        display(Markdown("Gleichung 2"))
        return (
            (sigma_sr - f_sy) ** 2
            * diam
            / (4 * E_sh * tau_b1 * s_r)
            * (1 - E_sh * tau_b0 / (E_s * tau_b1))
            + (sigma_sr - f_sy) / E_s * tau_b0 / tau_b1
            + (epsilon_sy - tau_b0 * s_r / (E_s * diam))
        )

    if (f_sy + 2 * tau_b1 * s_r / diam) <= sigma_sr <= f_su:
        display(Markdown("Gleichung 3"))
        return (sigma_sr - f_sy) / E_sh + (epsilon_sy - tau_b1 * s_r / (E_sh * diam))
    

# Extending the function to include the equations from Fall B (D.6 to D.10).
def esm_trilinear(sigma_max, f_sy, f_sh, fsu, E_s, E_sh1, E_sh2, tau_b0, tau_b1, tau_b2, diam, s_rm, epsilon_sy, epsilon_sh):
    # Fall A (D.1 to D.5) logic remains the same:
    # Equation D.1: for σ_max ≤ f_sy
    if (f_sh - f_sy)*diam/(4*tau_b1) > s_r/2:
        display(Markdown("Berechnungen nach @alvarez_einfluss_1998 Seite 163 Fall A"))
        if sigma_max <= f_sy:
            display(Markdown("Gleichung 1"))
            esm = (sigma_max / E_s) - (tau_b0 * s_rm) / (E_s * diam)
            return esm
        
        # Equation D.2: for f_sy < σ_max ≤ (f_sy + τ_b1 * s_rm / Ø)
        elif f_sy < sigma_max <= (f_sy + (tau_b1 * s_rm / diam)):
            display(Markdown("Gleichung 2"))
            term1 = (sigma_max - f_sy)**2 * diam / (4 * E_sh1 * tau_b1 * s_rm)
            term2 = (1 - (E_sh1 * tau_b0) / (E_s * tau_b1)) * tau_b0 / E_s
            term3 = epsilon_sy - (tau_b0 * s_rm) / (E_s * diam)
            esm = term1 + term2 + term3
            return esm
        
        # Equation D.3: for (f_sy + τ_b1 * s_rm / Ø) < σ_max ≤ f_sh
        elif (f_sy + (tau_b1 * s_rm / diam)) < sigma_max <= f_sh:
            display(Markdown("Gleichung 3"))
            esm = ((sigma_max - f_sy) / E_sh1) + epsilon_sy - (tau_b1 * s_rm) / (E_sh1 * diam)
            return esm
        
        # Equation D.4: for f_sh < σ_max ≤ (f_sh + τ_b2 * s_rm / Ø)
        elif f_sh < sigma_max <= (f_sh + (tau_b2 * s_rm / diam)):
            display(Markdown("Gleichung 4"))
            term1 = (sigma_max - f_sh)**2 * diam / (4 * E_sh2 * tau_b2 * s_rm)
            term2 = (1 - (E_sh2 * tau_b1) / (E_sh1 * tau_b2)) * tau_b1 / E_s
            term3 = epsilon_sy - (tau_b2 * s_rm) / (E_sh2 * diam)
            esm = term1 + term2 + term3
            return esm
        
        # Equation D.5: for (f_sh + 2 * τ_b2 * s_rm / Ø) ≤ σ_max ≤ f_su
        elif (f_sh + (2 * tau_b2 * s_rm / diam)) <= sigma_max <= fsu:
            display(Markdown("Gleichung 5"))
            esm = ((sigma_max - f_sh) / E_sh2) + epsilon_sy - (tau_b2 * s_rm) / (E_sh2 * diam)
            return esm
    
    elif (f_sh - f_sy)*diam/(4*tau_b1) <=s_r/2:
        display(Markdown("Berechnungen nach @alvarez_einfluss_1998 Seite 163 Fall B"))
        # Fall B (D.6 to D.10) logic:
        # Equation D.6: for σ_max ≤ f_sy
        if sigma_max <= f_sy:
            display(Markdown("Gleichung 1"))
            esm = (sigma_max / E_s) - (tau_b0 * s_rm) / (E_s * diam)
            return esm

        # Equation D.7: for f_sy < σ_max ≤ f_sh
        elif f_sy < sigma_max <= f_sh:
            display(Markdown("Gleichung 2"))
            term1 = ((sigma_max - f_sy)**2 * diam) / (4 * E_sh1 * tau_b1 * s_rm)
            term2 = (1 - E_sh1 * tau_b0 / (E_s * tau_b1)) * tau_b0 / E_s
            term3 = (sigma_max - f_sy) * tau_b0 / (E_s * tau_b1)
            term4 = epsilon_sy - tau_b0 * s_rm / (E_s * diam)
            esm = term1 + term2 + term3 + term4
            return esm
        
        # Equation D.8: for f_sh ≤ σ_max ≤ (f_sh + τ_b2 * s_rm / Ø)
        elif f_sh <= sigma_max <= (f_sh + 2 * tau_b2 * s_rm / diam - (f_sh - f_sy) * tau_b2 / tau_b1):
            display(Markdown("Gleichung 3"))
            term1 = (sigma_max - f_sh)**2 * diam / (4 * E_sh2 * tau_b2 * s_rm)
            term2 = (1 - (E_sh2 * tau_b0) / (E_s * tau_b1)) * tau_b0 / E_s
            term3 = (sigma_max - f_sh) * tau_b1 / (E_s * tau_b2)
            term4 = (epsilon_sy - epsilon_sh) / 2 * tau_b2 / tau_b1
            term5 = (epsilon_sy - tau_b0 * s_rm / (E_s * diam)) * tau_b0 / tau_b1
            esm = term1 + term2 + term3 + term4 + term5
            return esm
        
        # Equation D.9: for f_sh + τ_b2 * s_rm / Ø ≤ σ_max ≤ f_su
        elif (f_sh + 2 * tau_b2 * s_rm / diam - (f_sh - f_sy) * tau_b2 / tau_b1) <= sigma_max <= fsu:
            display(Markdown("Gleichung 4"))
            term1 = (sigma_max - f_sh)**2 * diam / (4 * E_sh2 * tau_b2 * s_rm)
            term2 = (1 - E_sh2 * tau_b1 / (E_sh1 * tau_b2)) * tau_b1 / E_s
            term3 = (sigma_max - f_sh) * tau_b1 / E_sh1
            term4 = (epsilon_sy - tau_b2 * s_rm / (E_sh2 * diam)) * tau_b2 / tau_b1
            esm = term1 + term2 + term3 + term4
            return esm
        
        # Equation D.10: for (f_sh + 2 * τ_b2 * s_rm / Ø) ≤ σ_max ≤ f_su
        elif (f_sh + 2 * tau_b2 * s_rm / diam) <= sigma_max <= fsu:
            display(Markdown("Gleichung 5"))
            esm = ((sigma_max - f_sh) / E_sh2) + epsilon_sh - (tau_b2 * s_rm) / (E_sh2 * diam)
            return esm
        
    # If none of the conditions are met, return an error.
    else:
        raise ValueError("σ_max is out of the valid range")

# This function is now extended to handle both Fall A and Fall B scenarios (D.1 to D.10).



### Plastischer Rotationswinkel

Der Plastische Roatationswinkel gibt die Rotation eines plastischen Gelenks voraus. 

![Modell zur Abschätzung der plastischen Rotation](../imgs/plast_rotation.png){#fig-plast_rot}

In [9]:
def theta_plastisch(l_pl, epsilon_sm, epsilon_sm_y, d, x):
    """berechnet den plastischen Rotationswinkel"""
    return l_pl * (epsilon_sm - epsilon_sm_y) / (d - x)

## Kaltverformter Betonstahl - B500A

### Eigenschaften des Betonstahls

Allgemein werden die Eigenschaften für den Betonstahl B500A mit einem Suffix $A$ gekennzeichnet.


In [10]:
# Fliessgrenze
f_sy_A = 570 * N / mm**2

# Zugfestigkeit
f_su_A = 600 * N / mm**2

# Fliessdehnung
varepsilon_sy_A = 2.78 * permille

# Bruchdehnung
varepsilon_su_A = 25 * permille

# Elastizitätsmodul
E_s_A = 205 * kN / mm**2

# Verfestigungsmodul
E_sh_A = 1350 * N / mm**2

put_out()

$$
\begin{aligned}
f_{sy A} & = 570.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & f_{su A} & = 600.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & \varepsilon_{sy A} & = 2.78 \ \mathrm{‰} \\ 
\varepsilon_{su A} & = 25 \ \mathrm{‰} \quad & E_{s A} & = 205.0 \ \frac{\mathrm{kN}}{\mathrm{mm}^{2}} \quad & E_{sh A} & = 1350.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \end{aligned}
$$

In [11]:
# | echo: false

stress_b500a_array = np.array([0, f_sy_A.m, f_su_A.m]) * MPa
strain_b500a_array = np.array([0, varepsilon_sy_A, varepsilon_su_A])

fig_b500a, ax_b500a = plt.subplots()
ax_b500a.plot(strain_b500a_array, stress_b500a_array.to(MPa).m, color="k")

# Scientific Notation if needed
apply_scientific_notation(ax_b500a, stress_b500a_array.m)

# Set axis labels
ax_b500a.set_ylabel(r"$\sigma$ [N/mm$^2$]")
ax_b500a.set_xlabel(r"$\varepsilon_s$ [-]")

# Set limits
ax_b500a.set_xlim(xmax=135 / 1000)
ax_b500a.set_ylim(ymax=750)

# Show the plot
# plt.tight_layout()
plt.savefig("imgs/jag_stress_strain_b500a.svg")
plt.close()

![Spannungs-Dehnungs-Beziehung des Betonstahls B500A](../imgs/jag_stress_strain_b500a_bearb.svg){#stress_strain_b500a}

### Linear elastische - gerissene Biegesteifigkeit

Wird fortan für sämtliche Verformungsberechnungen verwendet.
Querschnittsanalyse:


In [12]:
# Wertigkeit der Elastizitätsmodule
n = E_s_A / E_c

put_out()

$$
\begin{aligned}
n & = 7.0 \  \end{aligned}
$$

Die Druckzonenhöhe wird mittels dem Gleichgewicht der horizontalen Kräfte ermittelt

In [13]:
# Mittels Sympy (CAS für Python) wird eine analytische Lösung der
# Druckzonenhöhe für den gerissenen Querschnitt ermittelt.

# Summe der horizontalen Kräfte
sum_Fx = sp.sympify("1/2*x*b_w*sigma_c -sigma_s*a_s*b_w")

# Definition der Spannung in Abhängigkeit der Dehnungen
expr_sigma_c = sp.sympify("varepsilon_c * E_c")
expr_sigma_s = sp.sympify("varepsilon_s * n * E_c")

# Die Beziehung zwischen der Stahl- und Betondehnung nach Hook
expr_epsilon_c = sp.sympify("varepsilon_s /(d-x)*x")

# Substitution der Gleichungen
sum_Fx_subs = sum_Fx.subs(
    {
        "sigma_c": expr_sigma_c.subs("varepsilon_c", expr_epsilon_c),
        "sigma_s": expr_sigma_s,
    }
)

# Erstellen einer numerischen Funktion
x_solve = sp.solve(sum_Fx_subs, "x")[1]
x_solve_lambd = sp.lambdify(("a_s", "b_w", "n", "d"), x_solve)

In [14]:
# Druckzonenhöhe mit Parametern substituiert
x_II = x_solve_lambd(a_s, b_w, n, d_x).to(mm)

put_out()

$$
\begin{aligned}
x_{II} & = 47.91 \ \mathrm{mm} \end{aligned}
$$

In [15]:
# Flächenmoment 2. Grades für den Beton
I_cII = (b_w * x_II**3 / 12 + b_w * x_II * (x_II / 2) ** 2).to(mm**4)


# Flächenmoment 2. Grades für den Betonstahl
I_sII = (a_s * b_w * n * (d_x - x_II) ** 2).to(mm**4)

# Flächenmoment 2. Grades für den gerissenen Querschnitt
I_II = (I_cII + I_sII).to(mm**4)

put_out()

$$
\begin{aligned}
I_{cII} & = 36650532.72 \ \mathrm{mm}^{4} \quad & I_{sII} & = 244534349.46 \ \mathrm{mm}^{4} \quad & I_{II} & = 281184882.18 \ \mathrm{mm}^{4} \end{aligned}
$$

In [16]:
# Gerissene Biegesteifigkeit
EI_II = (E_c * I_II).to(kN * m**2)

put_out()

$$
\begin{aligned}
EI_{II} & = 8238.72 \ \mathrm{kN} \cdot \mathrm{m}^{2} \end{aligned}
$$

### Hebelarm der inneren Kräfte

Der Hebelarm der inneren Kräfte wird für den gesamten Träger als konstant angenommen.
Dabei wird dem Beton die Zylinderdruckfestigkeit hinterlegt, sowie erreicht der Stahl die Zugfestigkeit. 

In [17]:
# Druckzonenhöhe für den Biegewiderstand
x_dv_A = (a_s * b_w * f_su_A / (0.85 * f_cc * b_w)).to(mm)

# Hebelarm der inneren Kräfte für den Biegewiderstand
dv_A = d_x - 1 / 2 * 0.85 * x_dv_A

# Biegewiderstand
m_u_A = (a_s * f_su_A * dv_A).to(kNm / m)

put_out()

$$
\begin{aligned}
x_{dv A} & = 18.11 \ \mathrm{mm} \quad & dv_{A} & = 253.3 \ \mathrm{mm} \quad & m_{u A} & = 116.98 \ \frac{\mathrm{kNm}}{\mathrm{m}} \end{aligned}
$$

### Grenzzustände des Zugglieds

#### Risselement bei Fliessbeginn - Zustand $Z1$

Zunächst wird ein Grenzzustand betrachtet, bei dem das Risselement gerade beim Fliessbeginn steht.

In [18]:
# Stahlspannungen
sigma_1_sZ1 = f_sy_A
sigma_2_sZ1 = f_sy_A - 4 * tau_b0 * s_r / 2 / diam_x


# Stahldehnungen
varepsilon_1_sZ1 = sigma_1_sZ1 / E_s_A
varepsilon_2_sZ1 = sigma_2_sZ1 / E_s_A

put_out()

$$
\begin{aligned}
\sigma_{1 sZ1} & = 570.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & \sigma_{2 sZ1} & = 404.49 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & \varepsilon_{1 sZ1} & = 2.78 \ \frac{\mathrm{N}}{\mathrm{kN}} \\ 
\varepsilon_{2 sZ1} & = 1.97 \ \frac{\mathrm{N}}{\mathrm{kN}} \quad &  \quad &  
 \end{aligned}
$$

In [19]:
# Mittlere Stahldehnung nach 
varepsilon_m_sZ1 = esm_bilinear(E_s_A, E_sh_A, s_r, tau_b0, tau_b1, diam_x, f_sy_A, f_sy_A, f_su_A, varepsilon_sy_A).to(permille)

put_out()


Berechnungen nach @marti_tragverhalten_1999 Seite 156

Gleichung 1

$$
\begin{aligned}
\varepsilon_{m sZ1} & = 2.38 \ \mathrm{‰} \end{aligned}
$$

In [20]:
# Fliessmoment
m_y_A = (a_s * f_sy_A * dv_A).to(kNm / m)

put_out()

$$
\begin{aligned}
m_{y A} & = 111.13 \ \frac{\mathrm{kNm}}{\mathrm{m}} \end{aligned}
$$

In [21]:
# | echo: false


def plot_stress_strain(
    coordinates,
    filename,
    strain_mean=0,
    strain=False,
    stress=True,
):

    # Plot the polygon
    x, y = zip(*coordinates)  # Unzip the coordinates into x and y

    fig, ax = plt.subplots()

    if strain == True:
        stress = False
        xlabel = r"$x$ [mm]"
        ylabel = r"$\varepsilon$ [-]"
        # Plot the coordiantes
        ax.plot(x + (x[0],), y + (y[0],), "black")
        ax.fill(x, y, color="darkgray", alpha=0.2)
        ax.axhline(strain_mean, linestyle='--', color = 'black')

    if stress == True:
        xlabel = r"$x$ [mm]"
        ylabel = r"$\sigma$ [N/mm$^2$]"
        # Plot the coordiantes
        ax.plot(x + (x[0],), y + (y[0],), "black")

    # Scientific Notation if needed
    apply_scientific_notation(ax, np.array(y))

    # Add labels and show the plot
    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)

    # plt.show()
    # Save the plot to the specified file
    plt.savefig(filename)
    plt.close()

In [22]:
# | echo: false

coords_a1_stress = [
    (0, 0),
    (s_r.m / 2, 0),
    (s_r.m / 2, sigma_2_sZ1.m),
    (0, sigma_1_sZ1.m),
]

plot_stress_strain(coords_a1_stress, "imgs/jag_stress_a1.svg", stress=True)


coords_a1_strain = [
    (0, 0),
    (s_r.m / 2, 0),
    (s_r.m / 2, varepsilon_2_sZ1),
    (0, varepsilon_1_sZ1),
]

plot_stress_strain(coords_a1_strain, "imgs/jag_strain_a1.svg", strain=True,strain_mean=varepsilon_m_sZ1.to(los))

::: {#fig-jag_zustand_a1 layout-ncol=2}
![Spannungsverlauf entlang des Zugglieds auf halbem Rissabstand](../imgs/jag_stress_a1.svg){#fig-jag_stress_a1}

![Dehnungsverlauf entlang des Zugglieds auf halbem Rissabstand](../imgs/jag_strain_a1.svg){#fig-jag_strain_a1}

Spannungs- und Dehnungsverlauf entlang der Hälfte des Rissabstands für den Zustand $A1$
:::

#### Risselement beim Erreichen der Zugfestigkeit - Zustand $A2$

Innerhalb des Risselements befindet sich der Betonstahl über der Fliessgrenze, sowohl bereichsweise unterhalb der Fliessgrenze. Dies führt zu den folgenden Spannungs- und Dehnungsverteilungen.

In [23]:
# Plastischer Bereich
Delta_x_pl_Z2 = (f_su_A - f_sy_A) / (4 * tau_b1 / diam_x)

# Elastischer Bereich
Delta_x_el_Z2 = s_r / 2 - Delta_x_pl_Z2

put_out()

$$
\begin{aligned}
\Delta_{x pl Z2} & = 36.25 \ \mathrm{mm} \quad & \Delta_{x el Z2} & = 63.75 \ \mathrm{mm} \end{aligned}
$$

In [24]:
# Stahlspannungen
sigma_3_sZ2 = f_sy_A - 4 * tau_b0 * Delta_x_el_Z2 / diam_x
sigma_2_sZ2 = f_sy_A
sigma_1_sZ2 = f_sy_A + 4 * tau_b1 * Delta_x_pl_Z2 / diam_x

# Stahldehnung
varepsilon_1_sZ2 = varepsilon_su_A
varepsilon_2_sZ2 = varepsilon_sy_A
varepsilon_3_sZ2 = (sigma_3_sZ2 / E_s_A).to(permille)

put_out()

$$
\begin{aligned}
\sigma_{3 sZ2} & = 464.49 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & \sigma_{2 sZ2} & = 570.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & \sigma_{1 sZ2} & = 600.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \\ 
\varepsilon_{1 sZ2} & = 25 \ \mathrm{‰} \quad & \varepsilon_{2 sZ2} & = 2.78 \ \mathrm{‰} \quad & \varepsilon_{3 sZ2} & = 2.27 \ \mathrm{‰} \end{aligned}
$$

In [25]:
# Mittlere Stahldehnung
varepsilon_m_sZ2 = esm_bilinear(
    E_s_A, E_sh_A, s_r, tau_b0, tau_b1, diam_x, f_su_A, f_sy_A, f_su_A, varepsilon_sy_A
).to(permille)

put_out()

Berechnungen nach @marti_tragverhalten_1999 Seite 156

Gleichung 2

$$
\begin{aligned}
\varepsilon_{m sZ2} & = 6.64 \ \mathrm{‰} \end{aligned}
$$

In [26]:
# | echo: false

coords_Z2_stress = [
    (0, 0), 
    (Delta_x_pl_Z2.m, 0), 
    (s_r.m / 2, 0), 
    (s_r.m / 2, sigma_3_sZ2.m), 
    (Delta_x_pl_Z2.m, sigma_2_sZ2.m),
    (0, sigma_1_sZ2.m)
    ]

plot_stress_strain(coords_Z2_stress, "imgs/jag_stress_a2.svg", stress=True)


coords_Z2_strain = [
    (0, 0),
    (Delta_x_pl_Z2.m, 0),
    (s_r.m / 2, 0),
    (s_r.m / 2, varepsilon_3_sZ2),
    (Delta_x_pl_Z2.m, varepsilon_2_sZ2),
    (0, varepsilon_1_sZ2),
]

plot_stress_strain(coords_Z2_strain, "imgs/jag_strain_a2.svg", strain=True)

::: {#fig-jag_zustand_a2 layout-ncol=2}
![Spannungsverlauf entlang des Zugglieds auf halbem Rissabstand](../imgs/jag_stress_a2.svg){#fig-jag_stress_a2}

![Dehnungsverlauf entlang des Zugglieds auf halbem Rissabstand](../imgs/jag_strain_a2.svg){#fig-jag_strain_a2}

Spannungs- und Dehnungsverlauf entlang der Hälfte des Rissabstands für den Zustand $A2$
:::

### Systemanalyse 

Nun wird das System analysiert, sprich der Zweifeldträger.

#### Fliessen des Mittelauflagers - Zustand $S1$

In [27]:
# Die Streckenlast zum Erreichen des Fliessens bei B
q_S1_A = (8 * m_y_A*b_w / l**2).to(kN/m)

# maximale Verformung im Feld 
w_S1_A = (
    5 * q_S1_A * l**4 / (384 * EI_II) - m_y_A*b_w * l**2 / (16 * EI_II)
).to(mm)

put_out()

$$
\begin{aligned}
q_{S1 A} & = 13.89 \ \frac{\mathrm{kN}}{\mathrm{m}} \quad & w_{S1 A} & = 35.97 \ \mathrm{mm} \end{aligned}
$$

#### Fliessbeginn im Feld - Zustand $S2$

![Fliessen im Feld](../imgs/jag_fliessen_feld.png)

In [28]:
# Streckenlast beim Fliessbeginn im Feld
q_S2_A = (
    ((4 * m_y_A + 2 * m_u_A) / l**2 + 4 / l**2 * np.sqrt(m_y_A**2 + m_y_A * m_u_A))*b_w
).to(kN/m)

put_out()

$$
\begin{aligned}
q_{S2 A} & = 20.55 \ \frac{\mathrm{kN}}{\mathrm{m}} \end{aligned}
$$

##### Kontrolle der Duktilität

In [29]:
# Rotationswinkel (Verwendung der Arbeitsgleichung)
alpha_A = (
    (q_S2_A - q_S1_A) * l**3 / (24 * EI_II)
    - (m_u_A - m_y_A) * b_w * l / (3 * EI_II)
).to(deg)

# Verformungsbedarf
Theta_erf_S2_A = 2 * alpha_A

# Annahme für die Länge des Gelenks
l_pl = 2 * d_x

# Verformungsvermögen
Theta_pl_A = theta_plastisch(l_pl, varepsilon_m_sZ2, varepsilon_m_sZ1, d_x, x_dv_A).to(deg)

put_out()

$$
\begin{aligned}
\alpha_{A} & = 0.88 \ \mathrm{°} \quad & \Theta_{erf S2 A} & = 1.76 \ \mathrm{°} \quad & l_{pl} & = 522.0 \ \mathrm{mm} \\ 
\Theta_{pl A} & = 0.53 \ \mathrm{°} \quad &  \quad &  
 \end{aligned}
$$

Das Verformungsvermögen ist nicht ausreichend.

#### Ermittlung der Traglast

Da das Verformungsvermögen nicht ausreicht, wird mittels der maximalen Gelenkrotation die Traglast bestimmt.

In [30]:
# maximaler Rotationswinkel
alpha_u_A = Theta_pl_A / 2

# Streckenlast bis zum maximalen Rotationswinkel
Delta_q_A = (
    (alpha_u_A + (m_u_A - m_y_A) * l * b_w / (3 * EI_II)) * 24 * EI_II / l**3
).to(kN / m)

# Traglast für B500A
q_u_A = Delta_q_A + q_S1_A

put_out()

$$
\begin{aligned}
\alpha_{u A} & = 0.26 \ \mathrm{°} \quad & \Delta_{q A} & = 2.5 \ \frac{\mathrm{kN}}{\mathrm{m}} \quad & q_{u A} & = 16.39 \ \frac{\mathrm{kN}}{\mathrm{m}} \end{aligned}
$$

In [31]:
# maximale Verformung im Feld
w_u_A = (
    5 * q_u_A * l**4 / (384 * EI_II) - m_u_A * b_w * l**2 / (16 * EI_II)
).to(mm)

put_out()

$$
\begin{aligned}
w_{u A} & = 49.33 \ \mathrm{mm} \end{aligned}
$$

## Naturharter Stahl B500C

### Eigenschaften des Betonstahls

In [32]:
f_sy_C = 500 * N / mm**2  # Fliessgrenze
f_su_C = 600 * N / mm**2  # Zugfestigkeit
varepsilon_sy_C = 2.439 * permille  # Fliessdehnung
varepsilon_sh_C = 20 * permille  # Dehnung bei Verfestigungsbeginn
varepsilon_su_C = 125 * permille  # Bruchdehnung
E_s_C= 205 * kN / mm**2  # Elastizitätsmodul
E_sh_C = 952 * N / mm**2  # Verfestigungsmodul

put_out()

$$
\begin{aligned}
f_{sy C} & = 500.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & f_{su C} & = 600.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & \varepsilon_{sy C} & = 2.44 \ \mathrm{‰} \\ 
\varepsilon_{sh C} & = 20 \ \mathrm{‰} \quad & \varepsilon_{su C} & = 125 \ \mathrm{‰} \quad & E_{s C} & = 205.0 \ \frac{\mathrm{kN}}{\mathrm{mm}^{2}} \\ 
E_{sh C} & = 952.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad &  \quad &  
 \end{aligned}
$$

In [33]:
# | echo: false

stress_C_array = np.array([0, f_sy_C.m, f_sy_C.m, f_su_C.m]) * MPa
strain_C_array = np.array([0, varepsilon_sy_C, varepsilon_sh_C, varepsilon_su_C])


fig_b500c, ax_b500c = plt.subplots()

ax_b500c.plot(strain_C_array, stress_C_array.to(MPa).m, color="k")


# Set axis labels
ax_b500c.set_ylabel(r"$\sigma$ [N/mm$^2$]")
ax_b500c.set_xlabel(r"$\varepsilon_s$ [-]")

# Set limits
ax_b500c.set_xlim(xmax=125 / 1000)
ax_b500c.set_ylim(ymax=750)

# Show the plot
# plt.tight_layout()
plt.savefig("imgs/jag_stress_strain_b500c.svg")
plt.close()

![Spannungs-Dehnungs-Beziehung des Betonstahls B500C](../imgs/jag_stress_strain_b500c.svg){#stress_strain_b500c}

### Gernzzustände des Zugglieds

#### Risselement bei Fliessbeginn - Z3

In [34]:
# Stahlspannungen
sigma_1_sZ3 = f_sy_C
sigma_2_sZ3 = f_sy_C - 4 * tau_b0 * s_r / 2 / diam_x


# Stahldehnungen
varepsilon_1_sZ3 = (sigma_1_sZ3 / E_s_C).to(permille)
varepsilon_2_sZ3 = (sigma_2_sZ3 / E_s_C).to(permille)

put_out()

$$
\begin{aligned}
\sigma_{1 sZ3} & = 500.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & \sigma_{2 sZ3} & = 334.49 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & \varepsilon_{1 sZ3} & = 2.44 \ \mathrm{‰} \\ 
\varepsilon_{2 sZ3} & = 1.63 \ \mathrm{‰} \quad &  \quad &  
 \end{aligned}
$$

In [35]:
# Mittlere Stahldehnung beim Fliessen
varepsilon_m_sZ3 = esm_trilinear(f_sy_C, f_sy_C, f_sy_C, f_su_C, E_s_C, E_s_C*0, E_sh_C, tau_b0, tau_b1, tau_b1*0, diam_x, s_r, varepsilon_sy_C, varepsilon_sh_C).to(permille)

# Fliessmoment
m_y_A = (a_s * f_sy_C * dv_A).to(kN)
put_out()

Berechnungen nach @alvarez_einfluss_1998 Seite 163 Fall B

Gleichung 1

$$
\begin{aligned}
\varepsilon_{m sZ3} & = 2.04 \ \mathrm{‰} \quad & m_{y A} & = 97.48 \ \mathrm{kN} \end{aligned}
$$

### Versagensart des Querschnitts

In [36]:
# Stahlspannung zwischen den Rissen
sigma_su_3 = f_su_C - 4 * tau_b1 / diam_x * s_r / 2

# Stahldehnung zwischen den Rissen
varepsilon_su_A_3 = ((sigma_su_3 - f_sy_C) / E_sh_C + varepsilon_sh_C).to(permille)
varepsilon_sm = (varepsilon_su_A_3 + varepsilon_su_A) / 2

put_out()

$$
\begin{aligned}
\sigma_{su 3} & = 517.24 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & \varepsilon_{su A 3} & = 38.11 \ \mathrm{‰} \quad & \varepsilon_{sm} & = 31.56 \ \mathrm{‰} \end{aligned}
$$

In [37]:
# Betonstauchung
varepsilon_c = varepsilon_sm / (d_x - x_dv_A) * x_dv_A

varepsilon_cu

put_out()

$$
\begin{aligned}
\varepsilon_{c} & = 2.35 \ \mathrm{‰} \quad & \varepsilon_{cu} & = 5 \ \mathrm{‰} \end{aligned}
$$

Die Betonbruchstauchung wird überschritten. Das bedeutet, es stellt sich ein Betondruckversagen vor einem Zerreissen der Bewehrung ein.

In [38]:
# Mittlere Stahldehnung beim Druckversagen
varepsilon_sm_u = varepsilon_cu / x_dv_A * (d_x - x_dv_A)

# Maximale Stahldehnung beim Druckversagen
varepsilon_smax_u = varepsilon_su_A - (varepsilon_sm - varepsilon_sm_u)

# Maximale Stahlspannung beim Druckversagen
sigma_smax_u = f_su_C - (varepsilon_su_A - varepsilon_smax_u) * E_sh

# Maximaler Biegewiderstand
m_u = (a_s * sigma_smax_u * dv_A).to(kNm / m)
put_out()

NameError: name 'E_sh' is not defined

### Verformungsvermögen

In [38]:
# Plastischer Rotationswinkel, Rotationsbedarf für vollständige Umlagerung.
Theta_pl_max = (l_pl * (varepsilon_sm_u - varepsilon_sm_y) / (d_x - x_dv_A)).to(deg)

put_out()

$$
\begin{aligned}
\Theta_{pl max} & = 8.01 \ \mathrm{°} \end{aligned}
$$

### Fliessen beim Mittelauflager

In [39]:
# Streckenlast beim Fliessen des Mittelauflagers
q_y_comma_B = 8 * m_y_A / l**2 * b_w

# Verformung beim Fliessen des Mittelauflagers
w_m_y = (
    5 * q_y_comma_B * l**4 / (384 * EI_II) - m_y_A * b_w * l**2 / (16 * EI_II)
).to(mm)

put_out()

$$
\begin{aligned}
q_{y , B} & = 12.19 \ \frac{\mathrm{kN}}{\mathrm{m}} \quad & w_{m y} & = 31.55 \ \mathrm{mm} \end{aligned}
$$

### Fliessbeginn im Feld

#### Risselement beim Fliessen im Feld

In [40]:
# Im Feld wird das Fliessmoment erreicht
m_F = m_y_A

put_out()

$$
\begin{aligned}
m_{F} & = 97.48 \ \mathrm{kN} \end{aligned}
$$

In [41]:
# Fliessspannung wird vorausgesetzt

Delta_x_pl = 28.5 * mm
Delta_x_el_Z2 = s_r / 2 - Delta_x_pl


sigma_s_2 = 500 * N / mm**2
sigma_s_1 = sigma_s_2 + 4 * tau_b1 / diam_x * Delta_x_pl
sigma_s_3 = sigma_s_2 - 4 * tau_b0 / diam_x * Delta_x_el_Z2

put_out()

$$
\begin{aligned}
\Delta_{x pl} & = 28.5 \ \mathrm{mm} \quad & \Delta_{x el Z2} & = 71.5 \ \mathrm{mm} \quad & \sigma_{s 2} & = 500.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \\ 
\sigma_{s 1} & = 523.59 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & \sigma_{s 3} & = 381.66 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad &  
 \end{aligned}
$$

In [42]:
S_1 = center_trapez(sigma_s_2, sigma_s_1, 0 * MPa)
S_2 = center_trapez(sigma_s_3, sigma_s_2, 0 * MPa)

A_1 = area_trapez(sigma_s_1, sigma_s_2, Delta_x_pl)
A_2 = area_trapez(sigma_s_2, sigma_s_3, Delta_x_el_Z2)

NameError: name 'center_trapez' is not defined

In [39]:
sigma_sm = (A_1 * S_1 + A_2 * S_2) / (A_1 + A_2)


put_out()

$$
\begin{aligned}
\sigma_{sm} & = 232.56 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \end{aligned}
$$

In [40]:
# Dehnungen

varepsilon_1_sZ2 = ((sigma_s_1 - sigma_s_2) / E_sh + varepsilon_sh).to(permille)
varepsilon_s_21 = varepsilon_sh
varepsilon_s_22 = varepsilon_sy_A
varepsilon_3_sZ2 = (sigma_s_3 / E_s_A).to(permille)

S_1 = center_trapez(varepsilon_s_21, varepsilon_1_sZ2, 0)
S_2 = center_trapez(varepsilon_3_sZ2, varepsilon_s_22, 0)

A_1 = area_trapez(varepsilon_1_sZ2, varepsilon_s_21, Delta_x_pl)
A_2 = area_trapez(varepsilon_s_22, varepsilon_3_sZ2, Delta_x_el_Z2)

varepsilon_sm = (A_1 * S_1 + A_2 * S_2) / (A_1 + A_2)

put_out()

$$
\begin{aligned}
\varepsilon_{s 1} & = 44.77 \ \mathrm{‰} \quad & \varepsilon_{s 21} & = 20 \ \mathrm{‰} \quad & \varepsilon_{s 22} & = 2.44 \ \mathrm{‰} \\ 
\varepsilon_{s 3} & = 1.86 \ \mathrm{‰} \quad & S_{1} & = 16.98 \ \mathrm{‰} \quad & S_{2} & = 1.08 \ \mathrm{‰} \\ 
A_{1} & = 923.04 \ \mathrm{mm} \cdot \mathrm{‰} \quad & A_{2} & = 153.75 \ \mathrm{mm} \cdot \mathrm{‰} \quad & \varepsilon_{sm} & = 14.71 \ \mathrm{‰} \end{aligned}
$$

In [41]:
Theta_pl_q_y_comma_F = (l_pl * (varepsilon_sm - varepsilon_sm_y) / (d_x - x_dv_A)).to(
    deg
)

put_out()

$$
\begin{aligned}
\Theta_{pl q y , F} & = 1.56 \ \mathrm{°} \quad & deg & = \mathrm{°} \end{aligned}
$$

In [42]:
# | echo:false

ax_b500c.axvline(x=varepsilon_1_sZ2.to(los), color="black", linestyle="--")
ax_b500c.axhline(y=sigma_s_1.m, color="black", linestyle="--")

# Show the updated plot
# plt.tight_layout()
plt.figure(fig_b500c)
plt.savefig("imgs/jag_stress_strain_b500c_with_vertical_line.svg")
plt.close()

![Spannungs-Dehnungs-Verlauf des Betonstahls B500C mit markierter maximaler Dehnung](../imgs/jag_stress_strain_b500c_with_vertical_line.svg){#fig-stress_strain_b500c_marked}

In [43]:
# Stahlspannung durch maximale Stahldehnung am Zugglied
sigma_smax = (
    np.interp(
        x=varepsilon_1_sZ2.to(los), fp=stress_C_array.m, xp=strain_C_array
    )
    * N
    / mm**2
)

# Biegemoment resultiert daraus zu
m_B = (a_s * sigma_smax * dv_A).to(kN)

# Streckenlast Fliessen im Feld, verfestigend über dem Auflager
q_S2 = (
    (4 * m_F + 2 * m_B) / l**2 + 4 / l**2 * np.sqrt(m_F**2 + m_F * m_B)
) * b_w
put_out()

$$
\begin{aligned}
\sigma_{smax} & = 523.59 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & m_{B} & = 102.08 \ \mathrm{kN} \quad & q_{y , F} & = 18.0 \ \frac{\mathrm{kN}}{\mathrm{m}} \end{aligned}
$$

In [44]:
# Kontrolle des Verformungsbedarfs
alpha = (
    (q_S2 - q_y_comma_B) * l**3 / (24 * EI_II)
    - (m_B - m_y_A) * b_w * l / (3 * EI_II)
).to(deg)

# Erforderliche Rotation
Theta_pl_erf_i = 2 * alpha

# Kontrolle ob das Verformungsvermögen gleich dem Verformungsbedarf ist
Delta_Theta = Theta_pl_erf_i - Theta_pl_q_y_comma_F

put_out()

$$
\begin{aligned}
\alpha & = 0.78 \ \mathrm{°} \quad & \Theta_{pl erf i} & = 1.55 \ \mathrm{°} \quad & \Delta_{\Theta} & = -0.01 \ \mathrm{°} \end{aligned}
$$

In [45]:
# Die Mittendurchbiegung beträgt
w_m_y = (
    5 * q_S2 * l**4 / (384 * EI_II) - m_B * b_w * l**2 / (16 * EI_II)
).to(mm)

put_out()

$$
\begin{aligned}
w_{m y} & = 66.96 \ \mathrm{mm} \end{aligned}
$$

### Traglast 

In [46]:
varepsilon_2_sZ2 = varepsilon_sh
varepsilon_1_sZ2 = varepsilon_2_sZ2 + 4 * tau_b1 * s_r / 2 / (diam_x * E_sh)

varepsilon_sm_u = (varepsilon_2_sZ2 + varepsilon_1_sZ2) / 2
put_out()

$$
\begin{aligned}
\varepsilon_{s 2} & = 20 \ \mathrm{‰} \quad & \varepsilon_{s 1} & = 106.93 \ \mathrm{‰} \quad & \varepsilon_{sm u} & = 63.46 \ \mathrm{‰} \end{aligned}
$$

In [47]:
sigma_s_max = np.interp(varepsilon_1_sZ2, strain_C_array, stress_C_array)

m_B = m_u
m_F = (a_s * sigma_s_max * dv_A).to(kN)

# Streckenlast Fliessen im Feld, verfestigend über dem Auflager
q_u_comma_F = (
    (4 * m_F + 2 * m_B) / l**2 + 4 / l**2 * np.sqrt(m_F**2 + m_F * m_B)
) * b_w


# Plastische rotation im Feld
Theta_plf_F = (l_pl * (varepsilon_sm_u - varepsilon_sm_y) / (d_x - x_dv_A)).to(deg)


# Kontrolle des Verformungsbedarfs
alpha = (
    q_u_comma_F * l**3 / (24 * EI_II)
    - (m_B) * b_w * l / (3 * EI_II)
    + 0.414 * Theta_plf_F
).to(deg)

# Erforderliche Rotation
Theta_pl_erf = 2 * alpha

# Kontrolle ob das Verformungsvermögen gleich dem Verformungsbedarf ist
Delta_Theta = Theta_pl_erf - Theta_pl_max

put_out()

$$
\begin{aligned}
\sigma_{s max} & = 582.79 \ \mathrm{MPa} \quad & m_{B} & = 114.29 \ \frac{\mathrm{kNm}}{\mathrm{m}} \quad & m_{F} & = 113.62 \ \mathrm{kN} \\ 
q_{u , F} & = 20.73 \ \frac{\mathrm{kN}}{\mathrm{m}} \quad & \Theta_{plf F} & = 7.56 \ \mathrm{°} \quad & \alpha & = 4.09 \ \mathrm{°} \\ 
\Theta_{pl erf} & = 8.18 \ \mathrm{°} \quad & \Delta_{\Theta} & = 0.17 \ \mathrm{°} \quad &  
 \end{aligned}
$$

In [48]:
# Die Mittendurchbiegung beträgt
w_m_y = (
    5 * q_u_comma_F * l**4 / (384 * EI_II)
    - m_B * b_w * l**2 / (16 * EI_II)
    + 0.414 * Theta_plf_F * l / 2
).to(mm)

put_out()

$$
\begin{aligned}
w_{m y} & = 297.33 \ \mathrm{mm} \end{aligned}
$$